In [ ]:
# books_Excel.py
def showkind(url,kind,kindno):
    html = requests.get(url,headers=headers).text
    soup = BeautifulSoup(html,'html.parser')
    try:
        pages=int(soup.select('.cnt_page span')[0].text)  # 該分類共有多少頁
        if kindno==1:
            for page in range(4,pages+1):
                pageurl=url + '&page=' + str(page).strip()
                showpage(pageurl,kind,kindno,page)
        else:
            for page in range(1,pages+1):
                pageurl=url + '&page=' + str(page).strip()
                showpage(pageurl,kind,kindno,page)
    except:  # 沒有分頁的處理
        showpage(url,kind)

def showpage(url,kind,kindno,page):
    html = requests.get(url,headers=headers).text 
    soup = BeautifulSoup(html,'html.parser') 
    res = soup.find_all('div',{'class':'mod type02_m012 clearfix'})[0]
    if kindno==1 and page==4:
        items=res.select('.item')[21:]
    elif kindno==1 and page==5:
        items=res.select('.item')[:22]
    else:
        items=res.select('.item')  # 所有 item
    n=0  # 計算該分頁共有多少本書
    for item in items:
        msg=item.select('.msg')[0] 
        src=item.select('a img')[0]["src"]
        price_box=item.select('.price_box')[0]
        title=msg.select('a')[0].text  #書名
        publish=msg.select('a')[2].text #出版社
        date=msg.find('span').text.split("：")[-1] #出版日期
        discount=int(price_box.select('strong')[0].text) #折扣%
        if discount>10:
            discount=float(discount/10)
        onsale=int(price_box.select('strong')[-1].text) #優惠價
        try:
            price=round(int(price_box.select('strong')[1].text)/discount)*10
        except:
            price=onsale
        content=item.select('.txt_cont')[0].text.replace(" ","").strip()  #內容
        # 將資料加入 list1 串列中
        listdata=[title,publish,date,price,onsale,content]
        list1.append(listdata)
        n+=1

def twobyte(kindno):
    if kindno<10:
        kindnostr="0"+str(kindno)
    else:
        kindnostr=str(kindno) 
    return kindnostr

#主程式
import requests
from bs4 import BeautifulSoup
import csv
import openpyxl
import sqlite3
from time import sleep

list1=[]    
kindno=1
homeurl = 'https://www.books.com.tw/web/sys_bbotm/books/010201/?o=1&v=1&pd=5'
mode="/?o=1&v=1&pd=5"
mode1="/?o=3&v=1&bp=3"
url="https://www.books.com.tw/web/sys_bbotm/books/0102" 
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36'}
html = requests.get(homeurl,headers=headers).text
soup = BeautifulSoup(html,'html.parser') #中文書新書分類，取得分類資訊

res = soup.find('div',{'class':'mod type02_m012 clearfix'})
hrefs=res.select("a")
kindno=int(input("請輸入要下載的分類：(輸入1為散文，輸入2為小說)"))
if 0 < kindno <= len(hrefs):
    kind=hrefs[kindno-1].text #分類名稱
    print("下載的分類編號：{}  " .format(kindno))
    # 下載指定的分類
    if kindno==1:
        kindurl=url + twobyte(kindno) + mode # 分類網址
    else:
        kindurl=url + twobyte(kindno) + mode1
    showkind(kindurl,kind,kindno) # 顯示該分類所有書籍    
    # 儲存 cvs                                            
    print("資料寫入資料，請等侯幾分鐘!")
    workbook=openpyxl.Workbook()
    sheet = workbook.worksheets[0]
    listtitle=["書名","出版社","出版日期","定價","優惠價","內容"]
    sheet.append(listtitle)
    
    conn = sqlite3.connect('books.db')      #SQLite資料庫
    cursor = conn.cursor()
    sqlstr ='''CREATE TABLE IF NOT EXISTS books \
    ("書名"      TEXT PRIMARY KEY NOT NULL,
     "出版社"    TEXT NOT NULL,
     "出版日期"  TEXT NOT NULL,
     "定價"      INTEGER NOT NULL,
     "優惠價"    INTEGER NOT NULL,
     "內容"      TEXT NOT NULL
     )
    '''
    cursor.execute(sqlstr)
    
    with open('books.csv', 'w',encoding="utf-8-sig",newline='') as f:
        writer = csv.writer(f)
        writer.writerow(listtitle)
    
        for item1 in list1: #資料
            writer.writerow(item1)
            sheet.append(item1)
            sleep(0.1) # 必須加上適當的 delay
            conn.execute("INSERT INTO books (書名,出版社,出版日期,定價,優惠價,內容) VALUES \
                      ('{}', '{}', '{}', {}, {}, '{}')".format(item1[0], item1[1], item1[2], item1[3], item1[4], item1[5]))
        workbook.save('books.xlsx')

    conn.commit()
    conn.close()
    
    if kindno==1:
        count1,count2,count3=0,0,0
        for item1 in list1:
            if '2021-01' in item1[2]:
                count1+=1
            elif '2021-02' in item1[2]:
                count2+=1
            elif '2021-03' in item1[2]:
                count3+=1
        print("一月共出版", count1,"本")
        print("二月共出版", count2,"本")
        print("三月共出版", count3,"本")
        
    elif kindno==2:
        count1,count2,count3=0,0,0
        for item1 in list1:
            if 500<=item1[4] and item1[4]<=599:
                count1+=1
            elif 600<=item1[4] and item1[4]<=699:
                count2+=1
            elif 700<=item1[4] and item1[4]<=799:
                count3+=1
        print("500~599圓的書籍共", count1,"本")
        print("600~699圓的書籍共", count2,"本")
        print("700~799圓的書籍共", count3,"本")
        
    Publisher=[]
    for item1 in list1:
        if item1[1] not in Publisher:
            Publisher.append(item1[1])
    for i in range(len(Publisher)):
        count=0
        for item1 in list1:
            if Publisher[i]==item1[1]:
                count+=1
        print(Publisher[i],':',count)

else:
    print("分類不存在!")
print("")
print("資料儲存完畢!")    

In [13]:
#MySQL
#事先建立一個名為Books的資料庫
import pymysql
connmysql = pymysql.connect(host='localhost', port=3306, user='root', passwd='1234', charset='utf8',db='Books')

with connmysql.cursor() as cursor:
    sql = """
    CREATE TABLE IF NOT EXISTS Ranks ( 
      ID int not NULL AUTO_INCREMENT PRIMARY KEY,
      書名      varchar(100),
      出版社    varchar(50),
      出版日期  varchar(50),
      定價      int(5),
      優惠價    int(5),
      內容      varchar(1000)
    );
    """
    cursor.execute(sql)
    
    for item1 in list1:
        cursor.execute("INSERT INTO Ranks (書名,出版社,出版日期,定價,優惠價,內容) values \
                      ('{}', '{}', '{}', {}, {}, '{}')".format(item1[0], item1[1], item1[2], item1[3], item1[4], item1[5]))
    
    connmysql.commit()
connmysql.close()

In [103]:
#Google Sheet
import gspread
from oauth2client.service_account import ServiceAccountCredentials as sac
auth_json = 'stu93224-2021-1225-03cc7214fd19.json'
gs_scopes = ['http://spreadsheets.google.com/feeds']
cr = sac.from_json_keyfile_name(auth_json, gs_scopes)
gc = gspread.authorize(cr)
spreadsheet_key = '1C6vFNh24Bjxg-5Nf5XigS0EhksIW69m5wzIKdAtTHbU'
sheet = gc.open_by_key(spreadsheet_key)

wks = sheet.sheet1
wks.clear()
listtitle=["書名","出版社","出版日期","定價","優惠價","內容"]
wks.append_row(listtitle)
for item1 in list1:
    sleep(0.9)
    wks.append_row(item1)